## GeoTIFF to netCDF

---
Convert IMERG GeoTIFF in a folder to single NetCDF file with time dimension enabled that is CF-Compliant
http://cfconventions.org/cf-conventions/v1.6.0/cf-conventions.html
 
Based on Rich Signell's answer on StackExchange: https://gis.stackexchange.com/a/70487
This script was tested using IMERG dekad data. Adjustment is needed if using other timesteps data for IMERG
NCO (http://nco.sourceforge.net) must be installed before using this script
 
Modified by
Benny Istanto, UN World Food Programme, benny.istanto@wfp.org

---

### IMERG Global Dekad

In [1]:
#!/usr/bin/env python
import numpy as np
import datetime as dt
import os
import gdal
import netCDF4
import re

In [2]:
ds = gdal.Open('/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2000.06.3.tif')

In [3]:
a = ds.ReadAsArray()
nlat,nlon = np.shape(a)

In [4]:
b = ds.GetGeoTransform() #bbox, interval
lon = np.arange(nlon)*b[1]+b[0]
lat = np.arange(nlat)*b[5]+b[3]

basedate = dt.datetime(2000,1,1,0,0,0)

In [5]:
# create NetCDF file
nco = netCDF4.Dataset('wld_cli_imerg_1months_2000_2020_d3.nc','w',clobber=True)

# create dimensions, variables and attributes:
nco.createDimension('lon',nlon)
nco.createDimension('lat',nlat)
nco.createDimension('time',None)

timeo = nco.createVariable('time','f4',('time'))
timeo.units = 'days since 2000-1-1 00:00:00'
timeo.standard_name = 'time'
timeo.calendar = 'gregorian'
timeo.axis = 'T'

lono = nco.createVariable('lon','f4',('lon'))
lono.units = 'degrees_east'
lono.standard_name = 'longitude'
lono.long_name = 'longitude'
lono.axis = 'X'

lato = nco.createVariable('lat','f4',('lat'))
lato.units = 'degrees_north'
lato.standard_name = 'latitude'
lato.long_name = 'latitude'
lato.axis = 'Y'

# create container variable for CRS: lon/lat WGS84 datum
crso = nco.createVariable('crs','i4')
crso.long_name = 'Lon/Lat Coords in WGS84'
crso.grid_mapping_name='latitude_longitude'
crso.longitude_of_prime_meridian = 0.0
crso.semi_major_axis = 6378137.0
crso.inverse_flattening = 298.257223563

# create short integer variable for precipitation data, with chunking
pcpo = nco.createVariable('precip', 'f4',  ('time', 'lat', 'lon'),zlib=True,fill_value=-9999.)
pcpo.units = 'mm'
pcpo.standard_name = 'Precipitation rate'
pcpo.long_name = 'Monthly accumulated precipitation (combined microwave-IR) estimate'
pcpo.time_step = 'month'
pcpo.missing_value = -9999.
pcpo.geospatial_lat_min = -78.
pcpo.geospatial_lat_max = 82.3
pcpo.geospatial_lon_min = -180.
pcpo.geospatial_lon_max = 180.
pcpo.grid_mapping = 'crs'
pcpo.set_auto_maskandscale(False)

nco.Conventions='CF-1.6'
nco.title = "GPM IMERG Precipitation L3 1 month 0.1 degree x 0.1 degree"
nco.history = "The monthly rainfall data originally calculate from dekad, of which the dekad is accumulation of daily rainfall. Calculation process done using Cell Statistics of Spatial Analyst Toolbox in ArcGIS. The objective is to get monthly rainfall with rolling dekad accumulation."
nco.version = "Version 0.6"
nco.comments = "time variable denotes the first day of the given dekad."
nco.website = "https://doi.org/10.5067/GPM/IMERGDF/DAY/06"
nco.date_created = "2021-01-29"
nco.creator_name = "Benny Istanto"
nco.creator_email = "benny.istanto@wfp.org"
nco.institution = "UN World Food Programme"
nco.note = "The data is developed to support regular updating procedure for SPI analysis (https://github.com/wfpidn/SPI). This activities will support WFP to assess extreme dry and wet periods as part of WFP's Seasonal Monitoring"


#write lon,lat
lono[:]=lon
lato[:]=lat

pat = re.compile('wld_cli_imerg-v0.6.[0-9]{4}\.[0-9]{2}\.[0-9]{1}')
itime=0

#step through data, writing time and data to NetCDF
for root, dirs, files in os.walk('/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3'):
    dirs.sort()
    files.sort()
    for f in files:
        if re.match(pat,f):
            # read the time values by parsing the filename
            year=int(f[19:23])
            mon=int(f[24:26])
            dekad=int(f[27:28])
            date=dt.datetime(year,mon,dekad,0,0,0)
            print(date)
            dtime=(date-basedate).total_seconds()/86400.
            timeo[itime]=dtime
           # precipitation
            pcp_path = os.path.join(root,f)
            print(pcp_path)
            pcp=gdal.Open(pcp_path)
            a=pcp.ReadAsArray()  #data
            pcpo[itime,:,:]=a
            itime=itime+1

nco.close()

2000-06-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2000.06.3.tif
2000-07-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2000.07.3.tif
2000-08-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2000.08.3.tif
2000-09-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2000.09.3.tif
2000-10-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2000.10.3.tif
2000-11-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2000.11.3.tif
2000-12-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2000.12.3.tif
2001-01-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2001.01.3.tif
2001-02-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2001.02.3.tif
2001-03-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2001.03.3.tif
2001-04-03 00:00:00
/Users/ben

2007-07-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2007.07.3.tif
2007-08-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2007.08.3.tif
2007-09-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2007.09.3.tif
2007-10-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2007.10.3.tif
2007-11-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2007.11.3.tif
2007-12-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2007.12.3.tif
2008-01-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2008.01.3.tif
2008-02-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2008.02.3.tif
2008-03-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2008.03.3.tif
2008-04-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2008.04.3.tif
2008-05-03 00:00:00
/Users/ben

2014-09-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2014.09.3.tif
2014-10-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2014.10.3.tif
2014-11-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2014.11.3.tif
2014-12-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2014.12.3.tif
2015-01-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2015.01.3.tif
2015-02-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2015.02.3.tif
2015-03-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2015.03.3.tif
2015-04-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2015.04.3.tif
2015-05-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2015.05.3.tif
2015-06-03 00:00:00
/Users/bennyistanto/Temp/IMERG/Dekad/Dekad3/wld_cli_imerg-v0.6.2015.06.3.tif
2015-07-03 00:00:00
/Users/ben